In [19]:
import weaviate
import json

client = weaviate.Client(
    url = "",  # Replace with your endpoint
    auth_client_secret=weaviate.AuthApiKey(api_key=""),  # Replace w/ your Weaviate instance API key
)

In [5]:
!pip install -U sentence-transformers


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 1.9 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 49.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.2/36.2 MB 48.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 84.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.6/96.6 kB 29.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 42.0 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125925 sha256=1ab0e6d4aa6e28d54691a139f66107a72c0305c3bf1b01f775395f1b68a08467
  Stored in directory: /export/home/dhruv/.cache/pip/wheels/ff/27/bf/ffba8b318b02d7f691a57084ee154e26ed24d012b0c7805881
Successfully built sentence-transformers


In [20]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

def get_embedding(text):
    embeddings = model.encode(sentences)
    return embeddings


In [ ]:
class_obj = {
    "class": "Video_description",
    "vectorizer": "text2vec-huggingface",  # If set to "none" you must always provide vectors yourself. Could be any other "text2vec-*" also.
    "moduleConfig": {
        "text2vec-huggingface": {
            "model": "sentence-transformers/all-MiniLM-L6-v2",  # Can be any public or private Hugging Face model.
            "options": {
                "waitForModel": True
            }
        }
    }
}

client.schema.create_class(class_obj)

In [12]:
import os
import json
input_directory = 'output_data'

In [13]:
json_files = [f for f in os.listdir(input_directory) if f.endswith('.json')]


In [21]:
for file_name in json_files:
    input_file = os.path.join(input_directory, file_name)
    with open(input_file) as f:
        data = json.load(f)  
        metadata = data['metadata']['file']
        video_id = data['metadata']['text_id'][3:]
        with client.batch(batch_size=100) as batch:
            
            for sent in data['sentences']:
                embedding = model.encode(sent['sentence'])
                properties = {
                   "text": sent['sentence'],
                   "starttime" : sent['starttime'],
                   "endtime" : sent['endtime'],
                   "metadata" : metadata,
                   "video_id" : video_id
                }

                client.batch.add_data_object(
                    properties,
                    "Video_description",
                    vector = embedding
                )

In [26]:
import weaviate


client = weaviate.Client(
    url = "",  # Replace with your endpoint
    auth_client_secret=weaviate.AuthApiKey(api_key=""),  # Replace w/ your Weaviate instance API key
)

# Define the input sentence for which you want to find similar sentences
# input_sentence = "Donald trump after losing"

nearText = {"concepts": ["Donald trump after losing"]}

response = (
    client.query
    .get("Video_description", ["text", "starttime", "endtime", "metadata","video_id"])
    .with_near_text(nearText)
    .with_limit(5)
    .do()
)
print(json.dumps(response, indent=4))


{
    "data": {
        "Get": {
            "Video_description": [
                {
                    "endtime": "134.15",
                    "metadata": "/db/tv/2016/2016-02/2016-02-02/2016-02-02_1500_US_KABC_Good_Morning_America.txt",
                    "starttime": "131.23",
                    "text": "Big Questions Right now can donald trump bounce back from a loss",
                    "video_id": "4b9b3cd8_c9c6_11e5_b5ec_089e01ba0335"
                },
                {
                    "endtime": "231.40",
                    "metadata": "/db/tv/2016/2016-02/2016-02-02/2016-02-02_1500_US_KABC_Good_Morning_America.txt",
                    "starttime": "225.12",
                    "text": "Donald Trump called the republican front-runner for more than six months now has a new title loser .",
                    "video_id": "4b9b3cd8_c9c6_11e5_b5ec_089e01ba0335"
                },
                {
                    "endtime": "903.00",
                    "metadata":